# Confidence Intervals


This tutorial is going to demonstrate how to load data, clean/manipulate a dataset, and construct a confidence interval for the difference between two population proportions and means.

We will use the 2015-2016 wave of the NHANES data for our analysis.

*Note: We have provided a notebook that includes more analysis, with examples of confidence intervals for one population proportions and means, in addition to the analysis I will show you in this tutorial.  I highly recommend checking it out!

For our population proportions, we will analyze the difference of proportion between female and male smokers.  The column that specifies smoker and non-smoker is "SMQ020" in our dataset.

For our population means, we will analyze the difference of mean of body mass index within our female and male populations.  The column that includes the body mass index value is "BMXBMI".

Additionally, the gender is specified in the column "RIAGENDR".

In [61]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [62]:
# url = "nhanes_2015_2016.csv"
da = pd.read_csv('nhanes_2015_2016.csv')

In [63]:
da.columns

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')

## Investigating and Cleaning Data

In [64]:
da_new = da.copy()
da_new["SMQ020x"] = da_new.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})
da_new["SMQ020x"]

0       Yes
1       Yes
2       Yes
3        No
4        No
       ... 
5730    Yes
5731     No
5732    Yes
5733    Yes
5734     No
Name: SMQ020x, Length: 5735, dtype: object

In [65]:
da_new["RIAGENDRx"] = da_new.RIAGENDR.replace({1: "Male", 2: "Female"})
# da["RIAGENDRx"]

In [66]:
dx = da_new[["SMQ020x", "RIAGENDRx"]].dropna()
pd.crosstab(dx.SMQ020x, dx.RIAGENDRx)

RIAGENDRx,Female,Male
SMQ020x,,
No,2066,1340
Yes,906,1413


In [85]:
dz = dx.groupby("RIAGENDRx").agg({'SMQ020x': [np.mean, np.size]})
dz.columns = ['Proportion', 'Total N']
dz

,Proportion,Total N
RIAGENDRx,,
Female,0.304845,2972
Male,0.513258,2753


In [103]:
# dz.loc["Female", "Proportion"] # 0.30484522207267833

# dz.iloc[1, 1] # 2753

0.30484522207267833

### Constructing Confidence Intervals

Now that we have the population proportions of male and female smokers, we can begin to calculate confidence intervals.  From lecture, we know that the equation is as follows:

$$Best\ Estimate \pm Margin\ of\ Error$$

Where the *Best Estimate* is the **observed population proportion or mean** from the sample and the *Margin of Error* is the **t-multiplier**.

The equation to create a 95% confidence interval can also be shown as:

$$Population\ Proportion\ or\ Mean\ \pm (t-multiplier *\ Standard\ Error)$$

The Standard Error is calculated differenly for population proportion and mean:

$$Standard\ Error \ for\ Population\ Proportion = \sqrt{\frac{Population\ Proportion * (1 - Population\ Proportion)}{Number\ Of\ Observations}}$$

$$Standard\ Error \ for\ Mean = \frac{Standard\ Deviation}{\sqrt{Number\ Of\ Observations}}$$

Lastly, the standard error for difference of population proportions and means is:

$$Standard\ Error\ for\ Difference\ of\ Two\ Population\ Proportions\ Or\ Means = \sqrt{SE_{Proportion\ 1}^2 + SE_{Proportion\ 2} ^2}$$

#### Difference of Two Population Proportions

### Constructing Confidence Intervals

Now that we have the population proportions of male and female smokers, we can begin to calculate confidence intervals.  From lecture, we know that the equation is as follows:

$$Best\ Estimate \pm Margin\ of\ Error$$

Where the *Best Estimate* is the **observed population proportion or mean** from the sample and the *Margin of Error* is the **t-multiplier**.

The equation to create a 95% confidence interval can also be shown as:

$$Population\ Proportion\ or\ Mean\ \pm (t-multiplier *\ Standard\ Error)$$

The Standard Error is calculated differenly for population proportion and mean:

$$Standard\ Error \ for\ Population\ Proportion = \sqrt{\frac{Population\ Proportion * (1 - Population\ Proportion)}{Number\ Of\ Observations}}$$

$$Standard\ Error \ for\ Mean = \frac{Standard\ Deviation}{\sqrt{Number\ Of\ Observations}}$$

Lastly, the standard error for difference of population proportions and means is:

$$Standard\ Error\ for\ Difference\ of\ Two\ Population\ Proportions\ Or\ Means = \sqrt{SE_{Proportion\ 1}^2 + SE_{Proportion\ 2} ^2}$$

#### Difference of Two Population Proportions

In [105]:
dz.columns

Index(['Proportion', 'Total N'], dtype='object')

In [109]:
p_female = dz.loc['Female', "Proportion"]
n_female = dz.loc['Female', "Total N"]
se_female = np.sqrt(p_female * (1 - p_female)/n_female)
se_female

0.008444152146214435

In [110]:
p_male = dz.loc['Male', "Proportion"]
n_male = dz.loc['Male', "Total N"]
se_male = np.sqrt(p_male * (1 - p_male)/ n_male)
se_male

0.009526078653689868

In [111]:
se_diff = np.sqrt(se_female**2 + se_male**2)
se_diff

0.012729881381407434

In [112]:
d = p_female - p_male
lcb = d - 1.96 * se_diff
ucb = d + 1.96 * se_diff
(lcb, ucb)

(-0.2333636091471941, -0.18346247413207697)

#### Difference of Two Population Means

In [116]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

In [113]:
da["BMXBMI"].head()

0    27.8
1    30.8
2    28.8
3    42.4
4    20.3
Name: BMXBMI, dtype: float64

In [115]:
da.columns

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')

In [131]:
dm = da.groupby("RIAGENDRx").agg({"BMXBMI": [np.mean, np.std, np.size]}).droplevel(0, axis=1)
dm.columns

Index(['mean', 'std', 'size'], dtype='object')

In [32]:
male_mean = da[da["RIAGENDRx"]=='Male'].agg("BMXBMI").mean()
female_mean = da[da["RIAGENDRx"]=='Female'].agg("BMXBMI").mean()

male_std = da[da["RIAGENDRx"]=='Male'].agg("BMXBMI").std()
female_std = da[da["RIAGENDRx"]=='Female'].agg("BMXBMI").std()
male_size = len(da[da["RIAGENDRx"]=='Male'].agg("BMXBMI"))
female_size = len(da[da["RIAGENDRx"]=='Female'].agg("BMXBMI"))


In [33]:
sem_female =  female_std/ np.sqrt(female_size)
sem_male =  male_std / np.sqrt(male_size)
(sem_female, sem_male)

(0.14212522940758343, 0.1190371572233207)

In [19]:
sem_diff = np.sqrt(sem_female**2 + sem_male**2)
sem_diff

0.18538993598139303

In [133]:
dm_mean = dm.loc['Female', 'mean'] - dm.loc['Male', 'mean']

In [134]:
lcb = dm_mean - 1.96 * sem_diff
ucb = dm_mean + 1.96 * sem_diff
(lcb, ucb)

(0.7985092658034811, 1.5252378148505419)

In [135]:
def confint_two_mean():
    ...
    
def confint_two_proportions():
    ...

In [136]:
10**4

10000